# 内积神经网络

In [1]:
import mxnet
from mxnet import gluon
try:
    ctx = mxnet.gpu(0)
except _:
    ctx = mxnet.cpu()

/mnt/D/Ubuntu/package/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [3]:
import sys
sys.path.append("/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study")
from movielens.fm_basic import MyMutEmbedding

In [4]:
class InnerProductLayer(gluon.nn.Block):
    
    def __init__(self, **kwargs):
        super(InnerProductLayer, self).__init__(**kwargs)
        
    def forward(self, x):
        """
        
        :param x: 
        :return: 
        """
        y = mxnet.nd.swapaxes(x, 1, 2)
        result = mxnet.nd.batch_dot(x, y)
        result = mxnet.nd.flatten(result)
        return result

In [5]:
test_x = mxnet.nd.array([[[1, 2], [3, 4], [5, 6]], 
                         [[1, 1], [2, 2], [3, 3]], 
                         [[1, 4], [3, 2], [4, 4]], 
                         [[5, 2], [1, 4], [7, 2]]], ctx=ctx)
test_x


[[[ 1.  2.]
  [ 3.  4.]
  [ 5.  6.]]

 [[ 1.  1.]
  [ 2.  2.]
  [ 3.  3.]]

 [[ 1.  4.]
  [ 3.  2.]
  [ 4.  4.]]

 [[ 5.  2.]
  [ 1.  4.]
  [ 7.  2.]]]
<NDArray 4x3x2 @gpu(0)>

In [6]:
net = InnerProductLayer()


In [7]:
net(test_x)


[[  5.  11.  17.  11.  25.  39.  17.  39.  61.]
 [  2.   4.   6.   4.   8.  12.   6.  12.  18.]
 [ 17.  11.  20.  11.  13.  20.  20.  20.  32.]
 [ 29.  13.  39.  13.  17.  15.  39.  15.  53.]]
<NDArray 4x9 @gpu(0)>

In [54]:
train_x = mxnet.nd.array([[ 0.,  0.,  0.,  0.],
 [ 1.,  1.,  1.,  1.],
 [ 0.,  0.,  2.,  0.],
 [ 1.,  0.,  0.,  0.]])
train_y = mxnet.nd.array([ 1.,  0.,  1.,  1.])

In [223]:
input_dims = [3, 3, 4, 3]
output_dim = 5
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(MyMutEmbedding(input_dims, output_dim, is_reshape=True))
    net.add(InnerProductLayer())
    net.add(gluon.nn.Dense(10, activation='sigmoid'))
    net.add(gluon.nn.Dense(1, activation='sigmoid'))

In [224]:
net.collect_params()

sequential29_ (
  Parameter sequential29_mymutembedding0_embedding0_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential29_mymutembedding0_embedding1_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential29_mymutembedding0_embedding2_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential29_mymutembedding0_embedding3_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential29_dense0_weight (shape=(10, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential29_dense0_bias (shape=(10,), dtype=<class 'numpy.float32'>)
  Parameter sequential29_dense1_weight (shape=(1, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential29_dense1_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [225]:
net.initialize()

In [226]:
net(train_x)


[[ 0.48069388]
 [ 0.48069498]
 [ 0.48069465]
 [ 0.48069194]]
<NDArray 4x1 @cpu(0)>

In [227]:
loss = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)

In [228]:
trainer = gluon.Trainer(net.collect_params(), 'AdaGrad', {'learning_rate': 0.5})

In [229]:
from mxnet import autograd
for i in range(1000):
    with autograd.record():
        output = net(train_x)
        loss_num = loss(train_y, output) 
        if i % 100 == 0:
            print(mxnet.ndarray.sum(loss_num))
    loss_num.backward()
    trainer.step(4)


[ 56.32900238]
<NDArray 1 @cpu(0)>



[ 0.77899587]
<NDArray 1 @cpu(0)>



[ 0.36917219]
<NDArray 1 @cpu(0)>



[ 0.2412644]
<NDArray 1 @cpu(0)>



[ 0.17904633]
<NDArray 1 @cpu(0)>



[ 0.14229393]
<NDArray 1 @cpu(0)>



[ 0.11804029]
<NDArray 1 @cpu(0)>



[ 0.10084003]
<NDArray 1 @cpu(0)>



[ 0.08800959]
<NDArray 1 @cpu(0)>



[ 0.07807097]
<NDArray 1 @cpu(0)>


In [230]:
net(train_x)


[[ 0.99996459]
 [ 0.00243267]
 [ 0.99996459]
 [ 0.99996459]]
<NDArray 4x1 @cpu(0)>